<center><img src="../../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-25 12:28:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-25 12:28:02    

# Import the library for speech synthesis and broadcasting

In [ ]:
import time
import pygame 
from aip import AipSpeech


# """  APPID AK SK """
SpeechAPP_ID = '21548581'
SpeechAPI_KEY ='MgeoavC3XBKgOnhxU43uOOXZ'
SpeechSECRET_KEY = 'y95K6UCTvNypYlh8DC9WNcaW4VMl2y0G'


#connect client
Speechclient = AipSpeech(SpeechAPP_ID, SpeechAPI_KEY, SpeechSECRET_KEY)

#speech init
pygame.mixer.init()

def AudioPlay(text):
    result = Speechclient.synthesis(text, 'zh', 1, {'spd': 2, 'vol': 2, 'per': 1})  

    if not isinstance(result, dict):
        with open('./02.mp3', 'wb') as f:
            f.write(result)
        pygame.mixer.init()
        pygame.mixer.music.load('./02.mp3')
        pygame.mixer.music.play()
        time.sleep(2)

# import TensorFlow and camera library

In [ ]:
import numpy as np
import cv2
import os,time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from image_fun import bgr8_to_jpeg

# camera init

In [ ]:
# Init camera 
cap = cv2.VideoCapture(0)
cap.set(3, 640) # set Width
cap.set(4, 480) # set Height
cap.set(5, 5)  #set frame
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

# init tf model

In [ ]:
# Init tf model

MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09' #fast
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' 
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt') 
NUM_CLASSES = 90 
IMAGE_SIZE = (12, 8) 
fileAlreadyExists = os.path.isfile(PATH_TO_CKPT) 

if not fileAlreadyExists:
    print('Model does not exsist !')
    exit

In [ ]:
# LOAD GRAPH
print('Loading...')
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True) 
category_index = label_map_util.create_category_index(categories)
print('Finish Load Graph..')
print(type(category_index))
print("dict['Name']: ", category_index[1]['name'])

# main loop

In [ ]:
# Main
t_start = time.time()
fps = 0

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            ret, frame = cap.read()
#            frame = cv2.flip(frame, -1) # Flip camera vertically
#            frame = cv2.resize(frame,(160,120))
            ##############
            image_np_expanded = np.expand_dims(frame, axis=0) 
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
#             print('Running detection..') 
            (boxes, scores, classes, num) = sess.run( 
                [detection_boxes, detection_scores, detection_classes, num_detections], 
                feed_dict={image_tensor: image_np_expanded}) 

#             print('Done.  Visualizing..')
            vis_utils.visualize_boxes_and_labels_on_image_array(
                    frame,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)
            
            for i in range(0, 10):
                if scores[0][i] >= 0.5:
                    print('I find '+category_index[int(classes[0][i])]['name'])
                    AudioPlay('I find'+category_index[int(classes[0][i])]['name'])
            ##############
            
            fps = fps + 1
            mfps = fps / (time.time() - t_start)
            cv2.putText(frame, "FPS " + str(int(mfps)), (10,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            
            
            _, frame = cap.read()
            cv2.imshow('frame',frame)
             
            k = cv2.waitKey(3000) & 0xff
            if k == 27: # press 'ESC' to quit
                break
 
cap.release()